In [ ]:
!pip install sovai[full]

In [2]:
import sovai as sov
import pandas as pd

sov.token_auth(token="visit https://sov.ai/profile for your token")

In [3]:
tickers_meta = pd.read_parquet("data/tickers.parq")

In [4]:
df_short = sov.data("short/price_improvement")

In [1]:
import sovai as sov
import pandas as pd

sov.token_auth(token="visit https://sov.ai/profile for your token")

In [22]:
df_short_vol_hist = sov.data("short/volume", full_history=True)

In [26]:
df_short_vol_hist["short_ratio"] = df_short_vol_hist["short_volume"]/df_short_vol_hist["total_volume"]

In [27]:
df_short_vol_hist.query("ticker == 'AAPL'")

,ticker,date,short_volume,total_volume,short_volume_ratio_exchange,retail_short_ratio,institutional_short_ratio,market_maker_short_ratio,first_two_letters,short_ratio
9587,AAPL,2011-01-07,8893810.000,19134061.000,0.461,0.286,0.628,0.086,AA,0.465
9588,AAPL,2011-01-14,11789186.000,22008065.000,0.563,0.303,0.619,0.078,AA,0.536
9589,AAPL,2011-01-21,24197268.000,54387222.000,0.439,0.292,0.586,0.121,AA,0.445
9590,AAPL,2011-01-28,13856925.000,31652930.000,0.487,0.316,0.531,0.153,AA,0.438
9591,AAPL,2011-02-04,9718197.000,21816811.000,0.498,0.335,0.528,0.137,AA,0.445
...,...,...,...,...,...,...,...,...,...,...
10302,AAPL,2024-09-20,12089487.000,28928285.000,0.372,0.026,0.960,0.014,AA,0.418
10303,AAPL,2024-09-27,6880652.000,13460036.000,0.497,0.033,0.949,0.018,AA,0.511
10304,AAPL,2024-10-04,9262195.000,17907283.000,0.496,0.024,0.961,0.015,AA,0.517
10305,AAPL,2024-10-11,6715984.000,12796643.000,0.504,0.033,0.955,0.011,AA,0.525


In [24]:
df_short_vol_hist.tail()

,ticker,date,short_volume,total_volume,short_volume_ratio_exchange,retail_short_ratio,institutional_short_ratio,market_maker_short_ratio,first_two_letters
5582449,ZZZ,2024-09-20,6.000,6.000,1.000,0.000,1.000,0.000,ZZ
5582450,ZZZ,2024-09-27,233.000,237.000,0.981,0.000,1.000,0.000,ZZ
5582451,ZZZ,2024-10-04,950.000,961.000,0.750,NaN,NaN,NaN,ZZ
5582452,ZZZ,2024-10-11,6.000,515.000,0.333,NaN,NaN,NaN,ZZ
5582453,ZZZ,2024-10-18,1.000,1.000,1.000,0.000,1.000,0.000,ZZ


In [121]:
df_short_vol_levels = sov.data("short/volume")

df_short_vol_previous = sov.data("short/volume", frequency="previous")

df_short_vol_previous["short_ratio"] = df_short_vol_previous["short_volume"]/df_short_vol_previous["total_volume"]

df_short_vol_levels["short_ratio"] = df_short_vol_levels["short_volume"]/df_short_vol_levels["total_volume"]

# Select relevant columns from the previous DataFrame and rename short_ratio
df_prev = df_short_vol_previous[['ticker', 'short_ratio']].rename(columns={
    'short_ratio': 'sr_prev'  # short_ratio from previous date
})

# Select relevant columns from the current DataFrame and rename short_ratio
df_short_vol_levels = df_short_vol_levels.rename(columns={
    'short_ratio': 'sr_curr'  # short_ratio from current date
})

# Merge on 'ticker' using an inner join to include only tickers present in both DataFrames
df_short_vol_levels = pd.merge(df_short_vol_levels, df_prev, on='ticker', how='inner')


df_short_vol_levels[["short_volume","total_volume"]] = df_short_vol_levels[["short_volume","total_volume"]]/1000

df_short_vol_levels = df_short_vol_levels.rename(columns={"short_volume":"short_vol","total_volume":"tot_vol","short_volume_ratio_exchange":"sr_ex","retail_short_ratio":"sr_ret","institutional_short_ratio":"sr_int","market_maker_short_ratio":"sr_mm"})

df_short_vol_levels = df_short_vol_levels.dropna()

df_short_vol_levels = df_short_vol_levels.drop(columns=["first_two_letters"])

df_short_vol_levels["sr_diff"] = df_short_vol_levels["sr_curr"] - df_short_vol_levels["sr_prev"] 

df_short_vol_levels = df_short_vol_levels[["ticker","date","short_vol","tot_vol","sr_curr","sr_prev","sr_diff","sr_ex","sr_ret","sr_int","sr_mm"]]



In [122]:
df_short_vol_levels[["sr_curr","sr_prev","sr_diff","sr_ex","sr_ret","sr_int","sr_mm"]] = df_short_vol_levels[["sr_curr","sr_prev","sr_diff","sr_ex","sr_ret","sr_int","sr_mm"]]*100
df_short_vol_levels


,ticker,date,short_vol,tot_vol,sr_curr,sr_prev,sr_diff,sr_ex,sr_ret,sr_int,sr_mm
0,A,2024-10-18,830.493,2163.867,38.380,38.749,-0.369,39.331,1.173,98.571,0.257
1,AA,2024-10-18,4326.106,7716.817,56.061,55.012,1.049,55.454,0.965,98.753,0.282
2,AAA,2024-10-18,2.796,7.154,39.083,26.419,12.664,37.395,9.852,90.148,0.000
3,AAAU,2024-10-18,706.508,846.934,83.419,77.359,6.061,82.421,5.536,94.464,0.000
5,AACT,2024-10-18,22.552,25.965,86.855,15.448,71.407,56.368,5.633,94.367,0.000
...,...,...,...,...,...,...,...,...,...,...,...
9854,ZVSA,2024-10-18,47.458,86.331,54.972,68.580,-13.608,47.418,4.184,95.816,0.000
9855,ZWS,2024-10-18,2658.945,3671.824,72.415,60.007,12.407,69.874,0.477,99.503,0.020
9856,ZYME,2024-10-18,80.090,120.867,66.263,59.003,7.260,59.490,5.040,93.526,1.434
9857,ZYXI,2024-10-18,7.490,12.463,60.098,46.678,13.420,58.029,1.123,95.172,3.706


In [123]:
import sovai as sov
import pandas as pd


tickers_meta = pd.read_parquet("data/tickers.parq")

In [124]:
df_short_vol_levels = df_short_vol_levels.set_index(["ticker","date"])

df_short_vol_levels = df_short_vol_levels.filter(["market_cap>100"])

Condition 1: market_cap>100 (Standard filter)

Filtering Results:
┌─────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ Step            │   Total Tickers │         Removed │            Left │
┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼
│ Initial         │           7,929 │               - │               - │
│ Condition 1     │           3,612 │           4,317 │           3,612 │
┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼
│ Final           │           7,929 │           4,317 │           3,612 │
└─────────────────┴─────────────────┴─────────────────┴─────────────────┘


In [125]:
# Sort by 'sr_diff' and get top 30 and bottom 30, concatenate them
df_sorted = df_short_vol_levels.sort_values("sr_diff").reset_index()
df_top_bottom = pd.concat([df_sorted.head(30), df_sorted.tail(30)], ignore_index=True)

In [126]:
max_date = df_top_bottom["date"].max()
formatted_date = max_date.strftime('%B %d, %Y')
df_top_bottom = df_top_bottom.drop(columns=["date"])

In [ ]:
df_top_bottom = df_top_bottom.drop(columns=["sr_ret","sr_int","sr_mm"])

In [144]:
df_top_bottom = df_top_bottom.rename(columns={"sr_curr":"short_ratio"})

In [145]:
from datawrapper import Datawrapper

# Initialize Datawrapper with the access token
dw = Datawrapper(access_token="your_token")

# Create a new chart for short volume changes
chart2 = dw.create_chart(
    title="Short Volume Ratio Analysis",
    chart_type="tables"
)

# Add the data to the chart
dw.add_data(chart2['id'], data=df_top_bottom)

# Configure the visualization properties
metadata = {
    "visualize": {
        "columns": {
            "ticker": {
                "title": "Ticker", 
                "align": "left",
                "width": "100"
            },
            "sr_diff": {
                "title": "SR Change", 
                "format": "0.0%",
                "showAsBar": True,
                "width": 0.61,
                "fixedWidth": True,
                "style": {"color": False},
                "barColor": 7,
                "barColorNegative": 4
            },
            "short_vol": {
                "title": "Short Volume (K)", 
                "format": "0,0.0",
                "width": "120",
                "borderLeft": "none"
            },
            "tot_vol": {
                "title": "Total Volume (K)", 
                "format": "0,0.0",
                "width": "120",
                "borderLeft": "none"
            },
            "short_ratio": {
                "title": "Current SR", 
                "format": "0.0%",
                "width": "100",
                "style": {"color": "#c71e1d", "background": False},
                "borderLeft": "none"
            },
            "sr_prev": {
                "title": "Previous SR", 
                "format": "0.0%",
                "width": "100",
                "style": {"color": "#09bb9f"},
                "borderLeft": "none"
            },
            "sr_ex": {
                "title": "SR Expected", 
                "format": "0.0%",
                "width": "100",
                "borderLeft": "none"
            }
        },
        "dark-mode-invert": True,
        "sortBy": "sr_diff",
        "sortDirection": "asc",
        "perPage": 15,
        "pagination": {
            "enabled": True,
            "position": "bottom",
            "rowsPerPage": 15
        },
        "header": {
            "style": {
                "bold": True,
                "fontSize": 1.1,
                "background": False,
                "color": False,
                "italic": False
            },
            "borderTop": "none",
            "borderBottom": "2px",
            "borderTopColor": "#333333",
            "borderBottomColor": "#333333"
        },
        "showHeader": True,
        "markdown": False,
        "striped": False,
        "firstRowIsHeader": False,
        "firstColumnIsSticky": False,
        "mergeEmptyCells": False,

    },
    "describe": {
        "intro": "Analysis of largest short ratio changes. The table shows short volume, total volume, and short ratio (SR) metrics including current SR, previous SR, and the change between them.",
        "byline": "",
        "source-name": "Market Data",
        "source-url": "",
        "hide-title": False
    },
    "publish": {
        "embed-width": 648,
        "embed-height": 776,
        "blocks": {
            "logo": {"enabled": False},
            "embed": False,
            "download-pdf": False,
            "download-svg": False,
            "get-the-data": True,
            "download-image": False
        }
    }
}
# Update the chart with the metadata
dw.update_chart(
    chart_id=chart2['id'],
    metadata=metadata
)

# Publish the chart
dw.publish_chart(chart2['id']) 

# Get the URL of the published chart
chart_urls = dw.get_chart_display_urls(chart2['id'])
print("\nChart URLs:", chart_urls)


Chart URLs: [{'id': 'standalone', 'url': 'https://www.datawrapper.de/_/8bORt/', 'name': 'For sharing'}]


In [131]:
df_top_bottom

,ticker,short_vol,tot_vol,sr_curr,sr_prev,sr_diff,sr_ex
0,PDLB,1.468,9.275,15.827,86.386,-70.559,48.788
1,OBIO,4.471,16.187,27.621,90.302,-62.681,41.744
2,ALF,6.318,35.077,18.012,79.597,-61.585,72.427
3,NNOX,203.757,761.112,26.771,83.739,-56.968,70.233
4,ABUS,225.614,1742.273,12.949,69.217,-56.267,71.876
5,SYM,209.571,1277.839,16.400,67.009,-50.609,52.821
6,BWFG,1.782,7.981,22.328,71.812,-49.484,33.175
7,GCT,163.840,1086.956,15.073,61.092,-46.018,47.315
8,ARBE,11.962,43.553,27.465,71.817,-44.351,58.719
9,RUM,628.622,1569.450,40.054,84.368,-44.315,62.495


In [111]:
df_short_vol_levels.sort_values("sr_diff").head(50)

,,short_vol,tot_vol,sr_curr,sr_prev,sr_diff,sr_ex,sr_ret,sr_int,sr_mm
ticker,date,,,,,,,,,
PDLB,2024-10-18,1.468,9.275,0.158,0.864,-0.706,0.488,0.173,0.824,0.003
OBIO,2024-10-18,4.471,16.187,0.276,0.903,-0.627,0.417,0.050,0.950,0.000
ALF,2024-10-18,6.318,35.077,0.180,0.796,-0.616,0.724,0.000,1.000,0.000
NNOX,2024-10-18,203.757,761.112,0.268,0.837,-0.570,0.702,0.054,0.904,0.042
ABUS,2024-10-18,225.614,1742.273,0.129,0.692,-0.563,0.719,0.058,0.882,0.060
SYM,2024-10-18,209.571,1277.839,0.164,0.670,-0.506,0.528,0.043,0.947,0.010
BWFG,2024-10-18,1.782,7.981,0.223,0.718,-0.495,0.332,0.007,0.992,0.001
GCT,2024-10-18,163.840,1086.956,0.151,0.611,-0.460,0.473,0.045,0.943,0.012
ARBE,2024-10-18,11.962,43.553,0.275,0.718,-0.444,0.587,0.035,0.917,0.047


In [ ]:


def get_bankruptcy_date(frequency="difference"):

    df = sov.data("bankruptcy", frequency=frequency)
    
    df = df.set_index(["ticker","date"])
    
    df = df.filter(["market_cap>100"])
    
    df[["sans_market","probability"]].reset_index()
    
    df = pd.merge(df[["sans_market","probability"]].reset_index(),tickers_meta[["ticker","sector"]], on="ticker", how="left")
    
    df = df.rename(columns={"sans_market":"sansmarket"})
    return df


In [94]:
df_short_vol_levels.sample(20)

,ticker,short_vol,tot_vol,sr_curr,sr_prev,sr_diff,sr_ex,sr_ret,sr_int,sr_mm
3705,GIPR,0.830,3.836,0.216,0.445,-0.228,0.755,0.040,0.960,0.000
6680,PCF,48.524,106.931,0.454,0.347,0.106,0.508,0.020,0.980,0.000
9020,URE,3.409,6.334,0.538,0.521,0.017,0.624,0.000,1.000,0.000
5496,MAR,290.727,439.552,0.661,0.719,-0.058,0.680,0.035,0.963,0.002
7714,SBT,1.425,3.674,0.388,0.503,-0.115,0.432,0.201,0.799,0.000
3842,GQRE,3.852,11.725,0.329,0.604,-0.276,0.243,0.000,1.000,0.000
8945,UFPI,24.603,41.258,0.596,0.667,-0.071,0.613,0.060,0.919,0.020
6003,NAZ,9.579,14.126,0.678,0.473,0.205,0.707,0.071,0.929,0.000
8661,TKR,518.599,843.543,0.615,0.670,-0.055,0.763,0.031,0.967,0.002
8791,TRSG,0.657,0.857,0.767,0.791,-0.024,0.792,0.771,0.000,0.229


In [17]:
df_short_vol_levels["ratio"] =  df_short_vol_levels["short_volume"]  / df_short_vol_levels["total_volume"]

In [19]:
df_short_vol_levels["short_disparity"]  = df_short_vol_levels["short_volume_ratio_exchange"] -  df_short_vol_levels["ratio"]

In [21]:
df_short_vol_levels.sort_values("short_disparity")

,ticker,date,short_volume,total_volume,short_volume_ratio_exchange,retail_short_ratio,institutional_short_ratio,market_maker_short_ratio,first_two_letters,ratio,short_disparity
9808,XHYC,2024-10-18,2200.000,2224.000,0.248,NaN,NaN,NaN,XH,0.989,-0.741
6125,NBCC,2024-10-18,200.000,208.000,0.246,NaN,NaN,NaN,NB,0.962,-0.715
9654,WINC,2024-10-18,501.000,512.000,0.291,NaN,NaN,NaN,WI,0.979,-0.687
6738,PAB,2024-10-18,100.000,112.000,0.245,NaN,NaN,NaN,PA,0.893,-0.648
423,ANEB,2024-10-18,401.000,443.000,0.267,0.000,0.500,0.500,AN,0.905,-0.639
...,...,...,...,...,...,...,...,...,...,...,...
3510,FSZ,2024-10-18,102.000,2202.000,0.667,0.000,1.000,0.000,FS,0.046,0.621
1460,CAOS,2024-10-18,4689.000,23349.000,0.845,0.021,0.979,0.000,CA,0.201,0.644
1160,BNGE,2024-10-18,8.000,352.000,0.667,NaN,NaN,NaN,BN,0.023,0.644
3295,FJP,2024-10-18,23.000,1423.000,0.667,NaN,NaN,NaN,FJ,0.016,0.651


In [14]:
df_short_vol_change

,ticker,date,index,short_volume,total_volume,short_volume_ratio_exchange,retail_short_ratio,institutional_short_ratio,market_maker_short_ratio,first_two_letters
0,A,2024-10-18,-10041.000,118920.000,327481.000,-0.189,-0.007,0.019,-0.013,0
1,AA,2024-10-18,-10041.000,1037456.000,1738781.000,-0.066,0.002,0.001,-0.003,AA
2,AAA,2024-10-18,-10041.000,2326.000,5375.000,0.098,0.099,-0.099,0.000,AA
3,AAAU,2024-10-18,-10041.000,-419249.000,-608305.000,0.223,0.041,-0.041,0.000,AA
4,AACG,2024-10-18,-10041.000,2047.000,-1184.000,0.110,NaN,NaN,NaN,AA
...,...,...,...,...,...,...,...,...,...,...
10036,ZVSA,2024-10-18,-10058.000,45511.000,83492.000,-0.218,0.042,-0.042,0.000,ZV
10037,ZWS,2024-10-18,-10058.000,1487877.000,1720287.000,0.035,-0.002,0.006,-0.004,ZW
10038,ZYME,2024-10-18,-10058.000,20458.000,19801.000,-0.057,-0.008,0.076,-0.068,ZY
10039,ZYXI,2024-10-18,-10058.000,-6356.000,-17200.000,0.009,-0.002,0.135,-0.133,ZY


In [9]:
df_short_volume.dropna().sort_values("short_volume_ratio_exchange")

,ticker,date,index,short_volume,total_volume,short_volume_ratio_exchange,retail_short_ratio,institutional_short_ratio,market_maker_short_ratio,first_two_letters
2294,DFNM,2024-10-18,-10045.000,-41640.000,-30460.000,-0.802,0.001,-0.001,0.000,DF
6623,ONFO,2024-10-18,-10057.000,-601.000,1018.000,-0.734,-0.058,0.058,0.000,ON
7168,PSCX,2024-10-18,-10057.000,0.000,200.000,-0.667,0.000,0.000,0.000,PS
9852,XOCT,2024-10-18,-10057.000,-800.000,-430.000,-0.602,0.000,0.000,0.000,XO
4812,ITDE,2024-10-18,-10045.000,1149.000,3417.000,-0.602,0.000,0.000,0.000,IT
...,...,...,...,...,...,...,...,...,...,...
1965,CRD.B,2024-10-18,-10045.000,292.000,-1270.000,0.684,0.000,0.000,0.000,CR
4660,INFO,2024-10-18,-10046.000,417.000,-683.000,0.700,0.000,0.000,0.000,IN
6389,NULC,2024-10-18,-10057.000,164.000,64.000,0.709,0.000,0.000,0.000,NU
9812,XHYH,2024-10-18,-10057.000,1789.000,1806.000,0.759,0.163,-0.163,0.000,XH


In [7]:
df_short_volume

,ticker,date,index,short_volume,total_volume,short_volume_ratio_exchange,retail_short_ratio,institutional_short_ratio,market_maker_short_ratio,first_two_letters
0,A,2024-10-18,-10041.000,118920.000,327481.000,-0.189,-0.007,0.019,-0.013,0
1,AA,2024-10-18,-10041.000,1037456.000,1738781.000,-0.066,0.002,0.001,-0.003,AA
2,AAA,2024-10-18,-10041.000,2326.000,5375.000,0.098,0.099,-0.099,0.000,AA
3,AAAU,2024-10-18,-10041.000,-419249.000,-608305.000,0.223,0.041,-0.041,0.000,AA
4,AACG,2024-10-18,-10041.000,2047.000,-1184.000,0.110,NaN,NaN,NaN,AA
...,...,...,...,...,...,...,...,...,...,...
10036,ZVSA,2024-10-18,-10058.000,45511.000,83492.000,-0.218,0.042,-0.042,0.000,ZV
10037,ZWS,2024-10-18,-10058.000,1487877.000,1720287.000,0.035,-0.002,0.006,-0.004,ZW
10038,ZYME,2024-10-18,-10058.000,20458.000,19801.000,-0.057,-0.008,0.076,-0.068,ZY
10039,ZYXI,2024-10-18,-10058.000,-6356.000,-17200.000,0.009,-0.002,0.135,-0.133,ZY


In [62]:
df_short_volume = sov.data("short/volume")

UnboundLocalError: cannot access local variable 'data' where it is not associated with a value

In [61]:
df_short_volume.head()

short_volume  total_volume  short_volume_ratio_exchange  \
ticker date                                                                  
AAPL   2011-01-07   8893810.000  19134061.000                        0.461   
       2011-01-14  11789186.000  22008065.000                        0.563   
       2011-01-21  24197268.000  54387222.000                        0.439   
       2011-01-28  13856925.000  31652930.000                        0.487   
       2011-02-04   9718197.000  21816811.000                        0.498   

                   retail_short_ratio  institutional_short_ratio  \
ticker date                                                        
AAPL   2011-01-07               0.286                      0.628   
       2011-01-14               0.303                      0.619   
       2011-01-21               0.292                      0.586   
       2011-01-28               0.316                      0.531   
       2011-02-04               0.335                      0.528   

                   market_maker_short_ratio  
ticker date                                  
AAPL   2011-01-07                     0.086  
       2011-01-14                     0.078  
       2011-01-21                     0.121  
       2011-01-28                     0.153  
       2011-02-04                     0.137

In [140]:
dw.publish_chart("RUqwn")

{'data': {'publicId': 'RUqwn',
  'language': 'en-US',
  'theme': 'datawrapper',
  'id': 'RUqwn',
  'type': 'tables',
  'title': 'Short Volume Ratio Analysis',
  'lastEditStep': 5,
  'publishedAt': '2024-10-23T18:32:34.642Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/RUqwn/3/',
  'publicVersion': 3,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'upload-method': 'copy'},
   'describe': {'source-name': 'Market Data',
    'source-url': '',
    'intro': 'Analysis of largest short ratio changes. The table shows short volume, total volume, and short ratio (SR) metrics including current SR, previous SR, and the change between them.',
    'byline': '',
    'aria-description': '',
    'number-format': '-',
    'number-divisor': 0,
    'number-append': '',
    'number-prepend': '',
    'hide-title': False},
   'visualize': {'dark-mod

In [22]:
df_shorted.head()

,ticker,date,over_shorted,change,short_interest,actual,expected,dtc,market_cap
0,WRAP,2024-10-11,53.200,2.400,5237964.000,81.300,28.100,0.140,71.600
1,NLOP,2024-10-11,51.900,8.300,2371407.000,84.700,32.800,0.278,447.000
2,CPSS,2024-10-11,48.800,2.400,1135987.000,68.100,19.300,0.183,202.000
3,CNQ,2024-10-11,48.500,2.600,192390720.000,77.900,29.400,0.111,79680.102
4,GV,2024-10-11,48.500,-2.300,461139.000,81.000,32.600,0.488,7.200


In [3]:
### Here make space for some utility functions.
import os
from dotenv import load_dotenv
from github import Github
import plotly.graph_objects as go

def upload_file_to_github(plot_html, file_path_in_repo, commit_message="Add/update plot",
                         env_path='/Users/dereksnow/Sovai/GitHub/SovAI/.env', repo_name="sovai-research/sovai-research.github.io", git_token_env_var='GIT_TOKEN'):
    """
    Uploads or updates a file in the specified GitHub repository.

    Parameters:
    - plot_html (str): The content of the file to upload (e.g., HTML string).
    - file_path_in_repo (str): The path in the repository where the file will be uploaded (e.g., 'plots/risks/turing_risk_plot.html').
    - commit_message (str): The commit message. Defaults to "Add/update plot".
    - env_path (str, optional): The path to the .env file. If None, load_dotenv() will search automatically.
    - repo_name (str, optional): The full name of the repository (e.g., "owner/repo"). If None, read from environment variable 'GIT_REPO_NAME'.
    - git_token_env_var (str, optional): The name of the environment variable that holds the GitHub token. Defaults to 'GIT_TOKEN'.

    Returns:
    - None

    Raises:
    - ValueError: If the GitHub token or repository name is not provided.
    - Exception: For other unexpected errors during the upload/update process.
    """

    # Load environment variables
    if env_path:
        load_dotenv(dotenv_path=env_path)
    else:
        load_dotenv()  # Automatically load from .env in current or parent directories

    # Retrieve GitHub token from environment variables
    GIT_TOKEN = os.getenv(git_token_env_var)
    if not GIT_TOKEN:
        raise ValueError(f"{git_token_env_var} not found in environment variables. Please set it in the .env file or environment.")

    # Initialize GitHub client
    g = Github(GIT_TOKEN)

    # Retrieve repository name
    if not repo_name:
        repo_name = os.getenv('GIT_REPO_NAME')
        if not repo_name:
            raise ValueError("Repository name not specified and 'GIT_REPO_NAME' not found in environment variables.")

    # Initialize repository
    try:
        repo = g.get_repo(repo_name)
        print(f"Successfully connected to repository: {repo.full_name}")
    except Exception as e:
        print(f"Error accessing repository '{repo_name}': {e}")
        raise e

    # Upload or update the file
    try:
        # Attempt to get the existing file
        contents = repo.get_contents(file_path_in_repo)
        # If the file exists, update it
        repo.update_file(contents.path, commit_message, plot_html, contents.sha)
        print(f"File '{file_path_in_repo}' updated successfully.")
    except Exception as e:
        if "404" in str(e):
            # If the file does not exist, create it
            repo.create_file(file_path_in_repo, commit_message, plot_html)
            print(f"File '{file_path_in_repo}' created successfully.")
        else:
            print(f"An error occurred: {e}")
            raise e


In [4]:
df_congress = sov.data("congress")

In [21]:
import pandas as pd
from datetime import timedelta
import requests
import numpy as np

# Convert 'date' and 'transaction_date' to datetime
df_congress['date'] = pd.to_datetime(df_congress['date'])
df_congress['transaction_date'] = pd.to_datetime(df_congress['transaction_date'])

# Find the latest date
latest_date = df_congress['date'].max()
print(f"Latest Date: {latest_date.date()}")

# Calculate start date (7 days before latest_date)
start_date = latest_date - timedelta(days=7)
print(f"Start Date: {start_date.date()}")

# Filter for the last 7 days
df_last_7_days = df_congress[df_congress['date'] >= start_date].copy()
print(f"Number of Transactions in Last 7 Days: {df_last_7_days.shape[0]}")

# Get unique tickers in the last 7 days
unique_tickers = df_last_7_days['ticker'].unique()
print(f"Unique Tickers in Last 7 Days: {unique_tickers}")

# Define base URL for raw GitHub content
base_url = 'https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/'

# Define default logo URL
default_logo_url = 'https://avatars.githubusercontent.com/u/114351196?s=48&v=4'  # Replace with actual default if different

# Function to assign logo URL with error handling
def get_logo_url(ticker):
    url = f"{base_url}{ticker}.png"
    try:
        response = requests.head(url, timeout=5)
        if response.status_code == 200:
            return url
        else:
            return default_logo_url
    except requests.RequestException:
        return default_logo_url

# Create a dictionary mapping tickers to their logo URLs
ticker_logo_mapping = {ticker: get_logo_url(ticker) for ticker in unique_tickers}

# Display the mapping
for ticker, logo_url in ticker_logo_mapping.items():
    print(f"Ticker: {ticker}, Logo URL: {logo_url}")

# Assign logo URLs where 'date' >= start_date, else set to None
df_congress['logo'] = np.where(
    df_congress['date'] >= start_date,
    df_congress['ticker'].map(ticker_logo_mapping),
    None  # Or assign a default value like default_logo_url
)

# Alternatively, using pd.Series.where
# ticker_logo_series = pd.Series(ticker_logo_mapping)
# df_congress['logo'] = df_congress['ticker'].map(ticker_logo_series).where(df_congress['date'] >= start_date)

# Display the updated DataFrame
print(df_congress[['ticker', 'date', 'logo']])

# Filter congress dataframe based on date
df_congress = df_congress[df_congress['date'] >= start_date]

from datawrapper import Datawrapper
import pandas as pd

# Function to get price history for a ticker and convert to columns
def get_price_history_columns(ticker):
    try:
        # Get price data using your SDK
        df_price = sov.data("market/closeadj", tickers=[ticker])
        # Get last 90 days of data
        latest_prices = df_price.tail(90)
        # Convert dates to column names and values to a row
        return latest_prices['closeadj'].values
    except:
        return None

# Create a list to store the data
all_data = []

# Process each ticker
for idx, row in df_congress.iterrows():
    prices = get_price_history_columns(row['ticker'])
    if prices is not None:
        data_dict = {
            'ticker': row['ticker'],
            'logo': row['logo'],
            'transaction_date': row['transaction_date'],
            'representative': row['representative'],
            'bio_guide_url': row['bio_guide_url'],
            'transaction': row['transaction'],
            'amount': row['amount'],
            'house': row['house'],
            'transaction': row['transaction'],
            'party': row['party'],
            'days_to_report': row['days_to_report'],
        }
        # Add price columns
        for i, price in enumerate(prices):
            data_dict[f'price_{i}'] = price
        all_data.append(data_dict)

# Create new dataframe with price columns
df_with_prices_org = pd.DataFrame(all_data)


Latest Date: 2024-10-22
Start Date: 2024-10-15
Number of Transactions in Last 7 Days: 52
Unique Tickers in Last 7 Days: ['V' 'BX' 'BRK.B' 'TSLA' 'INTC' 'CAT' 'MSFT' 'HD' 'NSRGY' 'ASML' 'EFC'
 'IWM' 'SHY' 'MDY' 'IEF' 'IVV' 'FI' 'JMBS' 'EFA' 'VEA' 'FTGC' 'TLT' 'IEI'
 'TLH' 'WWSYX' 'PALL' 'TPYP' 'SCHW' 'CMBS' 'MBB' 'VNQI' 'TBLL' 'GIGB'
 'SCHP' 'EMLP' 'PPLT' 'AGZ' 'LSXMK' 'J' 'AMTM' 'CMG' 'NKE' 'ITW' 'ADP'
 'LSXMA' 'SIRI']
Ticker: V, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/V.png
Ticker: BX, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/BX.png
Ticker: BRK.B, Logo URL: https://avatars.githubusercontent.com/u/114351196?s=48&v=4
Ticker: TSLA, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/TSLA.png
Ticker: INTC, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/INTC.png
Ticker: CAT, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/CAT.png
Ticker: MSFT, L

In [47]:
df_with_prices = df_with_prices_org.copy()


# Format the data to include logos and links

# df_with_prices['ticker'] = df_with_prices.apply(
#     lambda row: f"{row['ticker']}<tooltip>{row['logo']}</tooltip>", 
#     axis=1
# )

df_with_prices['logo'] = df_with_prices['logo'].apply(lambda x: f'![logo]({x})')
df_with_prices['representative'] = df_with_prices.apply(
    lambda x: f'[{x["representative"]}]({x["bio_guide_url"]})', axis=1
)


df_with_prices['house_party'] = df_with_prices.apply(
    lambda x: f"{'Sen' if x['house']=='Senate' else 'Rep'} ({x['party']})", 
    axis=1
)


df_with_prices = df_with_prices.drop(columns=["bio_guide_url","party","house","days_to_report","logo"])

# # Format ticker with logo and Yahoo link
df_with_prices['ticker'] = df_with_prices.apply(
    lambda x: f"[{x['ticker']}](https://finance.yahoo.com/quote/{x['ticker']})", 
    axis=1
)

# # # Format ticker with logo and Yahoo link
# df_with_prices['transaction'] = df_with_prices.apply(
#     lambda x: f"[{x['transaction']}: {x['logo']}", 
#     axis=1
# )



# Get the list of column names
columns = df_with_prices.columns

# Filter for price_ columns
price_columns = [col for col in columns if col.startswith('price_')]

# Always keep the first and last price_ columns, and select every 4th column in between
selected_price_columns = [price_columns[0]] + price_columns[1:-1:4] + [price_columns[-1]]

# Define the final columns to keep, including the non-price columns
non_price_columns = [col for col in columns if not col.startswith('price_')]
final_columns = non_price_columns + selected_price_columns

# Create a new dataframe with the selected columns
df_with_prices = df_with_prices[final_columns]


df_with_prices = df_with_prices[[col for col in df_with_prices.columns if col not in ['house_party', 'transaction', 'amount',]] + ['house_party','transaction', 'amount']]

df_with_prices = df_with_prices.rename(columns={"transaction":"type", "transaction_date":"transaction","house_party":"house"})

# Display the result
df_with_prices.head()


,ticker,transaction,representative,price_0,price_1,price_5,price_9,price_13,price_17,price_21,price_25,price_29,price_33,price_37,price_41,price_45,price_49,price_53,price_57,price_61,price_65,price_69,price_73,price_77,price_81,price_85,price_89,house,type,amount
0,[V](https://finance.yahoo.com/quote/V),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,270.119,270.628,275.748,261.946,269.820,262.025,272.155,264.261,261.077,266.047,259.311,261.140,268.040,268.210,276.370,279.370,285.370,288.480,272.780,274.950,277.930,277.470,287.520,284.790,Rep (R),Purchase,1001.0
1,[BX](https://finance.yahoo.com/quote/BX),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,121.250,122.160,125.260,123.800,122.720,126.610,134.850,143.620,140.980,133.050,130.650,136.160,132.980,141.960,142.360,137.120,146.500,154.930,157.550,153.130,151.920,149.030,159.710,168.500,Rep (R),Purchase,1001.0
2,[BRK.B](https://finance.yahoo.com/quote/BRK.B),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,405.540,407.320,413.980,406.800,411.170,418.780,445.610,434.010,438.310,428.360,431.540,438.470,448.360,454.490,475.920,459.420,450.870,456.600,454.560,460.260,461.970,454.930,465.690,461.450,Rep (R),Purchase,1001.0
3,[TSLA](https://finance.yahoo.com/quote/TSLA),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,178.010,187.440,182.580,197.880,251.520,241.030,248.500,246.380,232.100,207.670,198.840,201.380,221.100,213.210,214.110,210.730,229.810,227.200,254.270,261.630,250.080,238.770,221.330,217.970,Rep (R),Purchase,1001.0
4,[INTC](https://finance.yahoo.com/quote/INTC),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,30.251,30.777,30.370,30.767,31.811,33.281,34.235,32.735,30.628,21.340,20.490,19.920,20.990,20.130,22.040,18.890,19.360,20.770,22.810,23.460,22.590,23.220,22.310,22.400,Rep (R),Purchase,1001.0


In [51]:
# Initialize Datawrapper
dw = Datawrapper(access_token="your_token")

# Create the chart
chart = dw.create_chart(
    title="Congressional Stock Trades with Price History",
    chart_type="tables"
)

# Add the data to the chart
dw.add_data(chart['id'], data=df_with_prices)

# Get price column names
price_cols = [col for col in df_with_prices.columns if col.startswith('price_')]

# Configure the visualization properties
properties = {
    "visualize": {
        "dark-mode-invert": True,
        "perPage": 20,
        "columns": {
            "ticker": {
                "align": "left",
                "title": "Stock",
                "tooltip": {
                    "enabled": True,
                    "template": "{{logo}}"
                }
            },
            "transaction": {
                "title": "Date",
                "format": "YYYY-MM-DD"
            },
            "representative": {
                "title": "Representative",
                "width": 0.23,
                "markdown": True,
                "fixedWidth": True
            },
            "price_0": {  
                "type": "number",
                "title": "Price History",
                "width": 0.28,
                "format": "$.2f",
                "visible": True,
                "sparkline": {
                    "color": "#18a1cd",
                    "title": "stock_price",
                    "enabled": True,
                    "stroke": 2,
                    "dotMax": True,
                    "dotMin": True,
                    "dotFirst": True,
                    "dotLast": True
                },
                "fixedWidth": True
            },
            "type": {
                "title": "Type",
                "customColor": True,
                "customColorBy": "type",
                "customColorText": {
                    "Sale": "#ee493a",
                    "Exchange": "#18a1cd",
                    "Purchase": "#09bb9f",
                    "Sale (Partial)": "#ffb239"
                }
            },
            "amount": {
                "title": "Amount",
                "format": "$0,0",
                "align": "right",
                "showAsBar": True,
                "fixedWidth": True
            },
            "house": {
                "title": "Chamber"
            }
        },
        "header": {
            "style": {
                "bold": True,
                "fontSize": 0.9,
                "color": "#494949"
            },
            "borderBottom": "2px",
            "borderBottomColor": "#333333"
        },
        "pagination": {
            "enabled": True,
            "position": "bottom",
            "pagesPerScreen": 10
        },
        "striped": True,
        "markdown": True,
        "showHeader": True,
        "compactMode": True,
        "firstRowIsHeader": False,
        "firstColumnIsSticky": False,
        "mergeEmptyCells": False
    }
}

# Configure remaining price columns
for col in price_cols[1:]:
    properties["visualize"]["columns"][col] = {
        "type": "number",
        "width": 0.28,
        "format": "$.2f",
        "visible": True,
        "sparkline": {
            "color": "#18a1cd",
            "title": "stock_price",
            "enabled": True
        },
        "fixedWidth": True,
        "showOnMobile": False,
        "showOnDesktop": True
    }

# Set column order
properties["visualize"]["column-order"] = [
    "ticker",
    "transaction",
    "representative",
    "price_0"
] + price_cols[1:] + [
    "type",
    "amount",
    "house"
]

# Add other visualization settings
properties["describe"] = {
    "intro": "Congressional stock trades with 90-day historical price performance.",
    "byline": "",
    "source-name": "Congress Trading Data",
    "source-url": "",
    "hide-title": False
}

properties["publish"] = {
    "embed-width": 682,
    "embed-height": 1086,
    "blocks": {
        "logo": {"enabled": False},
        "embed": False,
        "download-pdf": False,
        "download-svg": False,
        "get-the-data": True,
        "download-image": False
    },
    "autoDarkMode": False,
    "chart-height": 988,
    "force-attribution": False
}

# Update the chart with the properties
dw.update_chart(
    chart['id'],
    metadata=properties
)

# Publish the chart
dw.publish_chart(chart['id'])

# Get the published URL
published_url = dw.get_chart_display_urls(chart['id'])
print("Published Chart URL:", published_url)

Published Chart URL: [{'id': 'standalone', 'url': 'https://www.datawrapper.de/_/cv4aP/', 'name': 'For sharing'}]
